# MOTSynth_3 tester
Here we try to take a frame from a MOTSynth video, applying data visualization (bounding boxes, instances, class).
Since it is no clear how the coordinates are annotated (real world? or relative respect camera?) let's try to plot instance's coordinates in a 3d graph,
to see how they are referenced. 

NOTE: MOT webpage says coordinates are in world coordinates, but on annotations we don't have camera coordinates/rotation :( 

In [6]:
import os
from tkinter import *
from matplotlib import pyplot as plt
import matplotlib
matplotlib.use('TkAgg',force=True)
import numpy as np
import cv2

from PeopleDetector.utils.plots import plot_one_box
from PeopleDetector.utils.datasets import LoadStreams, LoadImages

In [2]:
class MOTline:
    def __init__(self, frame:int, id:int,bb_left:int, bb_top:int, bb_width:int, bb_height:int, 
    visibility:float, class_:int, confidence:float, x:float, y:float, z:float):
        self.frame_ = frame
        self.id_ = id
        self.bb_left_ = bb_left
        self.bb_top_ = bb_top
        self.bb_width_ = bb_width
        self.bb_height_ = bb_height
        self.visibility = visibility,
        self.class_ = class_
        self.confidence_ = confidence
        self.x_ = x
        self.y_ = y
        self.z_ = z

class MOTannotation:
    def __init__(self, path:str):
        annotations = open(path, 'r')
        self.data_ = {}
        for line in annotations:
            l = line.strip().split(',')
            v = MOTline(int(l[0]),int(l[1]),int(l[2]),int(l[3]),int(l[4]),int(l[5]),float(l[6]),int(l[7]),float(l[8]),float(l[9]),float(l[10]),float(l[11]))
            if not v.frame_ in self.data_.keys():
                self.data_[v.frame_] = []
            self.data_[v.frame_].append(v)
        annotations.close()

    def __getitem__(self, frame:int):
        return self.data_[frame]


In [3]:
data_folder = os.path.join(".", "inputs", "videos", "512")
video_path = os.path.join(data_folder, "512.mp4")
annotations_path = os.path.join(data_folder, "gt", "gt.txt")
target_frame = 3

data = MOTannotation(annotations_path)

In [8]:
#let's show the frame with all bounding box
#matplotlib.use('Agg',force=True)
video = LoadImages(video_path,1920)

frame_counter = 0
for path, img, im0s, vid_cap in video:
    frame_counter+=1
    if(frame_counter < target_frame):
        continue
    
    for el in data[target_frame]:
        xyxy = el.bb_left_, el.bb_top_, el.bb_left_ + el.bb_width_, el.bb_top_ + el.bb_height_
        #print(xyxy)
        plot_one_box(xyxy, im0s, label=str(el.id_), line_thickness=4)
    break
print(im0s.shape)

gbr = im0s[...,[2,1,0]]#.copy()
plt.imshow(gbr)
plt.show()

video 1/1 (1/1800) d:\Data\University\People-Analysis\inputs\videos\512\512.mp4: 
video 1/1 (2/1800) d:\Data\University\People-Analysis\inputs\videos\512\512.mp4: 
video 1/1 (3/1800) d:\Data\University\People-Analysis\inputs\videos\512\512.mp4: 
(1080, 1920, 3)


In [9]:
#let's show the 3d position of people in first frame.
fig = plt.figure()
ax = fig.add_subplot(projection='3d')

for el in data[target_frame]:
    ax.scatter(el.x_, el.y_, el.z_)
    ax.text(el.x_,el.y_,el.z_,  '%s' % (str(el.id_)), size=10, zorder=1,  color='k')


ax.scatter(0,0,0 , marker='^')
ax.text(0,0,0,  '_O_', size=10, zorder=1,  color='k')
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

plt.show()